# Transition Probs Iterator script

In [1]:
import sumProduct as sp
import numpy as np
import graphviz as gp
import subprocess
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
factors=[] 
variables=[]
hidden=[]
sensory=[]
allNodes=[]

In [3]:
vlist=['P'+str(i) for i in range(20)]
hidden=[sp.variableNode(name, vrange=['pain','nopain']) for name in vlist]
len(hidden)

20

In [4]:
Slist=['S'+str(i) for i in range(20)]
sensory=[sp.variableNode(name, vrange=['tickle','noci']) for name in Slist]
variables=hidden+sensory
print(len(variables))

40


In [5]:
pT0=sp.factorNode()
pT0.addNeighbour(hidden[0])
# Kann man den prior hier als 'situationsspezifischen' Prior verstehen? 
def PT0(x):
    T1,=x
    if T1=='pain':
        return 0.3
    if T1=='nopain':
        return 0.7
    raise ValueError('Invalid argument'+str(x))
    
pT0.setValues(PT0)
factors+=[pT0]
print(len(factors))

1


In [6]:
def pTgivenT(x):
    T1,T2=x
    if T1=='pain' and T2=='pain':
        return 0.9889531994561856 #Pp=0.3, Pnp=0.7
    if T1=='pain' and T2=='nopain':
        return 1-0.9558127978247422
    if T1=='nopain' and T2=='pain':
        return 1-0.9889531994561856
    if T1=='nopain' and T2=='nopain':
        return 0.9558127978247422
    raise ValueError('Invalid argument'+str(x))

def pSgivenT(x):
    S1,T1=x
    if S1=='tickle' and T1=='nopain':
        return 0.9
    if S1=='noci' and T1=='nopain':
        return 0.1
    if S1=='tickle' and T1=='pain':
        return 0.1
    if S1=='noci' and T1=='pain':
        return 0.9
    raise ValueError('Invalid argument'+str(x))

In [7]:
parent=hidden[0]

for child in hidden[1:]:
        p=sp.factorNode()
        p.addNeighbour(child)
        p.addNeighbour(parent)
        p.setValues(pTgivenT)
        factors+=[p]
        parent=child

child = sensory[0]

for h,s in zip(hidden,sensory):
    f=sp.factorNode()
    f.addNeighbour(s)
    f.addNeighbour(h)
    factors+=[f]
    f.setValues(pSgivenT)

print(len(factors))

allNodes=variables+factors

40


In [8]:
sp.runSumProduct(allNodes)

In [9]:
Marginals=[]
for h in hidden: 
    Marginals.append(h.marginal()[0])

In [10]:
pain_prob = [d['pain'] for d in Marginals]
print(pain_prob)
nopain_prob = [d['nopain'] for d in Marginals]
print(nopain_prob)

[0.3, 0.3276170013595362, 0.35370860519088715, 0.3783590653052723, 0.4016479818386731, 0.4236505582929438, 0.4444378443795127, 0.46407696544985394, 0.4826313392535958, 0.500160880724211, 0.5167221954535743, 0.5323687624801445, 0.547151106981025, 0.5611169634255495, 0.574311429717243, 0.5867771128219045, 0.5985542663520679, 0.6096809205521233, 0.6201930051038388, 0.6301244651488416]
[0.7, 0.6723829986404638, 0.6462913948091129, 0.6216409346947278, 0.5983520181613269, 0.5763494417070562, 0.5555621556204873, 0.535923034550146, 0.5173686607464042, 0.4998391192757889, 0.48327780454642566, 0.4676312375198555, 0.4528488930189751, 0.4388830365744505, 0.42568857028275703, 0.4132228871780956, 0.40144573364793207, 0.39031907944787664, 0.37980699489616127, 0.36987553485115837]
